### Description : 
##### Given a property decription , find out amenities from description

### Approach :
##### 1. Training a Custom NER
##### 2. This requires the following step : a) Create train data in the format of training , the data is first created in a json format and then transformed to the format as spacy require
##### 3. Documentation explored for training custom NER: https://spacy.io/usage/training , https://medium.com/analytics-vidhya/custom-named-entity-recognition-ner-model-with-spacy-3-in-four-steps-7e903688d51

### Evaluation :
##### 1. From each data point we find how many amenities are matching with the given list of amenities and depending on that we find score for each row and average them overall , the value range from 0-1 , more the score more would be the accuracy of the model.
##### 2. Second approach of measurement is , for each amenity in the amenities predicted , find out if same or similar amenity is present in the actual list , like in approach 1 , more the score more would be the accuracy.

### Other Approaches : 
#### 1. Training Custom NER with other methods 

### Challenges with this approach :
##### 1. Interpretability of the method 
##### 2. Require large amount of data for training
##### 2. Data Challenges : Amenities mentioned in the given data are not mentioned at all sometimes in the description. Hard to evaluate how this rule based method performs.
##### 3. Need more clarity , experimentation and exploration of this method


### Importing Libraries

In [1]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

In [2]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm
from spacy.tokens import DocBin

In [3]:
df = pd.read_csv('assignment_hotel_training.csv')

In [4]:
df.head()

,hotel_ameneties,property_description
0,Hot & Cold water|Credit cards accepted|Airpor...,New Soul Kiss Group of Houseboats |New Soul Ki...
1,Hot/cold Water |Parking Facility| Shower Area...,Hotel Raj International |Hotel Raj Internation...
2,Backup generator|Indoor Multi Cuisine Restaura...,Avisa Nila Beach Resort | Avisa Nila Beach Res...
3,Business centre|WiFiBathroom Toiletries |Baker...,Mayflower Hotel |Occupying a prime location on...
4,Dining Table|Geyser In Bathroom|Bathroom Toile...,The Zara Residency Hotel Located in the heart ...


### Processing sentences : Removing punctuations, extra white spaces, stripping, lowerinf, remove stopwords and lemmatize 

In [6]:
 remove = string.punctuation
regex = re.compile('[%s]' % re.escape(remove))

def pre_processing_words(sent):
    ## Removing punctuations, extra spaces, striping and lowering, removing stop words, lemmatize
    new_sent = regex.sub(' ', sent)
    new_sent = re.sub(' +',' ',new_sent).strip().lower()
    text_tokens = new_sent.split()
    tokens_without_sw = [lemmatizer.lemmatize(word) for word in text_tokens if not word in stopwords.words('english')]
    final_sent = ' '.join(tokens_without_sw)
    
    return final_sent

### Processing amenities

In [7]:
def split_sentences(string_to_iterate):
    all_sents = []
    char_index = 0

    while char_index < len(string_to_iterate)-1:        
        if (string_to_iterate[char_index].islower() and string_to_iterate[char_index+1].isupper()) or (string_to_iterate[char_index]==')' and string_to_iterate[char_index+1].isupper()) or (string_to_iterate[char_index].islower() and string_to_iterate[char_index+1].isdigit()) or (string_to_iterate[char_index]== ')' and string_to_iterate[char_index+1].isdigit()):
            all_sents.append(string_to_iterate[:char_index+1])
            string_to_iterate = string_to_iterate[char_index+1:]
            char_index = 0
        else:
            char_index = char_index+1

    if len(string_to_iterate):
        all_sents.append(string_to_iterate)

    
    return all_sents


def process_amenities(hotel_amenities):
    all_amenities = hotel_amenities.split('|')
    all_amenities = [amen.strip() for amen in all_amenities]
    
    new_amen_list = []
    for amen in set(all_amenities):
        processed_new_list = []
        all_words = split_sentences(amen)

        i = 0
        while i < len(all_words):
            if all_words[i] == 'Wi' and 'Fi'in all_words[i+1]:
                processed_new_list.append(all_words[i]+all_words[i+1])
                del all_words[i+1]
                i = i+1
            else:
                processed_new_list.append(all_words[i])
                i = i+1

        new_amen_list.append(processed_new_list)
    

    new_amen_list = list(set([pre_processing_words(amen) for sublist in new_amen_list for amen in sublist]))  
    
    return new_amen_list


### Processing property description

In [8]:
def process_description(desc):
    all_desc = desc.split('|')
    all_desc = [re.sub(r'\.(?!\d)', '_',d).split('_') for d in all_desc]
    all_desc_sentences = [pre_processing_words(a).strip() for sublist in all_desc for a in sublist]
    all_desc_sentences = [i for i in all_desc_sentences if i != '']
    return all_desc_sentences

### Finding amenities in description with indexes for each sentence in description

In [9]:
def tagging_amenities(amenity_list,list_description):
    all_res = {}
    for description in list_description:
        res = {key: [description.index(key), description.index(key) + len(key) - 1]
           for key in amenity_list if ' '+key+'' in description}
        all_res[description] = res

    return all_res

### Applying all processing methods to the dataframe

In [10]:
df['processed_amenities'] = df['hotel_ameneties'].apply(lambda x : process_amenities(x))
df['processed_description'] = df['property_description'].apply(lambda x : process_description(x))
df['tagged_data'] = df.apply(lambda x : tagging_amenities(x['processed_amenities'],x['processed_description']),axis=1)


### Creating json train file which would further be required to make train data in the format required to train custom NER

In [11]:
def create_training_data_format(tag_data):
    all_train_data = []
    for key in tag_data:
        if tag_data[key] == {}:
            continue
        else:
            final_keys = []
            for k in tag_data[key]:
                final_keys.append((tag_data[key][k][0],tag_data[key][k][1],'AMEN'))
            
            c = {'entities' : final_keys}
            train_data = (key,c)
            all_train_data.append(train_data)
            
    return all_train_data

### Splitting train/validation set [90% and 10%] and creating train and validation data in json format 

In [12]:
train_df = df.sample(frac = 0.90)
validation_df = df.drop(train_df.index)

train_df = train_df.reset_index(drop = True)
validation_df = validation_df.reset_index(drop = True)

In [13]:
train_df['train_data_format'] = train_df.tagged_data.apply(lambda x : create_training_data_format(x))

In [14]:
TRAIN_DATA = [val for sublist in train_df['train_data_format'].to_list() for val in sublist]

In [16]:
validation_df['train_data_format'] = validation_df.tagged_data.apply(lambda x : create_training_data_format(x))
validation_data = [val for sublist in validation_df['train_data_format'].to_list() for val in sublist]

### Training Custom NER

In [18]:
## Data Preparation for Training

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object
for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end+1, label=label)
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    print(ents)     
    try:
        doc.ents = ents # label the text with the ents
        db.add(doc)
    except:
        print(text, annot)
         
    
db.to_disk("./train.spacy") # save the docbin object

  1%|▏                                     | 123/20975 [00:00<00:32, 641.86it/s]

[swimming pool, outdoor activity, health club]
[restaurant]
[game room, guide service]
[television, safe deposit locker]
[gymnasium]
[doctor call]
[room heater]
[shower area]
[restaurant]
[telephone]
[restaurant]
[room service, doctor call]
[telephone]
[garden]
[dining table]
[mineral water, daily newspaper, room heater]
[wi fi internet]
[restaurant]
[tea coffee maker, direct dial phone, study table, television, mini bar]
[hot cold water]
[room service]
[attached bathroom]
[swimming pool, restaurant]
[library]
[doctor call]
[refrigerator]
[hot cold running water]
[restaurant]
[restaurant]
[parking facility, laundry service, taxi service, guide service, doctor call]
[room service, daily newspaper, hot cold water, bathroom toiletry, mineral water]
[restaurant]
[bar]
[swimming pool]
[coffee shop]
[garden]
[travel desk]
[laundry service, doctor call, outdoor activity]
[doctor call]
[taxi service, laundry]
[study table, hot cold water, mineral water, bathroom toiletry, shower area]
[backup 

  2%|▌                                     | 316/20975 [00:00<00:23, 886.33it/s]


[fitness centre]
[restaurant]
[coffee shop]
[room service]
[television, intercom]
[safety deposit box, luggage storage, 24 hour front desk]
[restaurant]
[bar, parking facility, taxi service, laundry service, air conditioning, doctor call, bathroom toiletry, airport transfer, restaurant]
[restaurant]
[parking facility]
[laundry, garden]
[garden]
[attached bathroom, television, telephone, bathroom toiletry]
[room service, doctor call]
[telephone]
[hot cold water]
[restaurant]
[bar, direct dial phone, safety deposit box, wi fi internet, mini bar, doctor call, refrigerator, parking facility, intercom facility, snack basket, travel desk, data port, tea coffee maker, daily newspaper, fi internet]
room amenity include cable television intercom facility direct dial phone mini bar tea coffee maker mineral wter refrigerator snack basket daily newspaper wi fi internet amenity request includes dvd player safety deposit box data port iron ironing board fruit basket hotel close proximity yercaud 30

  3%|▉                                     | 529/20975 [00:00<00:20, 983.96it/s]

...


[restaurant]
[attached bathroom, hot cold water]
[boutique]
[room service, railway station transfer, bonfire, outdoor activity, bicycle hire, laundry, garden, travel desk, internet facility, dining hall, wifi enabled]
[study table, dining table, sofa unit, intercom facility, wifi internet, bathroom toiletry, data port, balcony sit, tea coffee maker]
[restaurant]
[coffee shop]
[yoga class, yoga]
[trekking, tree house, jeep safari]
[bathroom toiletry, tea coffee maker]
[parking facility, conference hall, restaurant]
[direct dial phone, study table, iron ironing board request, air conditioning, intercom facility, dvd player request]
[room service, parking facility, guide service, travel desk, dining hall]
[taxi service]
[restaurant]
[laundry, front desk, travel assistance]
[business centre]
[restaurant]
[television, telephone]
[hot cold water]
[telephone]
[hot cold running water]
[business centre, lounge]
[lounge]
[table tennis, tennis court, child play area, chess, garden]
[travel desk,

 99%|███████████████████████████████████▋| 20770/20975 [00:22<00:00, 987.63it/s]

[restaurant]
[spa]
Skipping entity
[]
Skipping entity
[]
Skipping entity
[]
Skipping entity
[]
[swimming pool]
[mini refrigerator, direct dial phone, electronic safe, refrigerator]
amenity like lcd tv direct dial phone electronic safe mini refrigerator wi fi chargeable provided room {'entities': [(54, 70, 'AMEN'), (20, 36, 'AMEN'), (38, 52, 'AMEN'), (59, 70, 'AMEN')]}
[game room, swimming pool]
[restaurant]
[coffee shop]
[front desk]
[hot cold water]
[car rental]
[hot cold water]
[laundry service]
[room service, doctor call, travel desk, swimming pool]
[bar, mini bar, electronic safe, telephone, television, bathroom toiletry, tea coffee maker, attached bathroom]
host modern amenity television telephone tea coffee maker mini bar electronic safe attached bathroom various essential bathroom toiletry provided room ensure comfortable stay guest {'entities': [(63, 65, 'AMEN'), (58, 65, 'AMEN'), (67, 81, 'AMEN'), (31, 39, 'AMEN'), (20, 29, 'AMEN'), (119, 135, 'AMEN'), (41, 56, 'AMEN'), (83, 9

100%|████████████████████████████████████| 20975/20975 [00:22<00:00, 933.95it/s]



[secretarial service]
[restaurant]
[coffee shop]
[travel assistance]
[doctor call]
[microwave, television, refrigerator]
[kitchen]
[intercom facility, mineral water, daily newspaper]
[doctor call, parking facility]
[conference hall]
[restaurant]
[hot cold water, telephone]
[laundry service, railway station transfer, doctor call, taxi service, airport transfer, business center, conference hall, dining hall]
[safe deposit locker, study table, bath tub, newspaper, mini bar, air conditioning, electronic safe, dining table, refrigerator, hot cold water, dvd player, mineral water, snack basket, satellite television, piped channel music, fruit basket, geyser bathroom, bathroom toiletry, tea coffee maker, daily newspaper, room heater, shower area, living room]
hotel kingston 22 well rendered room consisting air conditioning living room cable satellite television dvd player piped channel music intercom direct dial telephone tea coffee maker mineral water wi fi internet access refrigerator frui

In [19]:
## Data prep for Validation

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object
for text, annot in tqdm(validation_data): # data in previous format
    doc = nlp(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end+1, label=label)
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
            
    try:
        doc.ents = ents # label the text with the ents
        db.add(doc)
    except:
        print(text, annot)
         
    
db.to_disk("./dev.spacy") # save the docbin object

 12%|████▋                                 | 289/2336 [00:00<00:01, 1476.76it/s]

Skipping entity
Skipping entity
23 standard 8 suite room hotel well facilitated satellite television room service telephone separate seating area attached bathroom hot cold water supply powered solar water heating system {'entities': [(48, 67, 'AMEN'), (69, 80, 'AMEN'), (132, 145, 'AMEN'), (58, 67, 'AMEN'), (82, 90, 'AMEN')]}
hotel offer facility like wi fi access pick drop facility air conditioning parking facility laundry service doctor call power back newspaper {'entities': [(92, 106, 'AMEN'), (131, 139, 'AMEN'), (58, 73, 'AMEN'), (108, 118, 'AMEN'), (92, 98, 'AMEN')]}
Skipping entity
amidst beautiful landscape contemporary interior hotel feature various modern facility suchlike wi fi internet access doctor call laundry adventure activity sightseeing trekking travel desk parking {'entities': [(118, 128, 'AMEN'), (130, 136, 'AMEN'), (96, 116, 'AMEN'), (157, 167, 'AMEN'), (138, 155, 'AMEN'), (178, 188, 'AMEN'), (185, 188, 'AMEN')]}
Skipping entity
golden plaza hotel 4 km chandigarh ai

 26%|██████████                            | 618/2336 [00:00<00:01, 1599.61it/s]


feature 39 well furnished elegant room basic comfort like air conditioning cable satellite television intercom direct dial phone mini bar mineral water refrigerator daily newspaper safety deposit locker room heater attached bathroom geyser bathroom toiletry {'entities': [(111, 127, 'AMEN'), (58, 73, 'AMEN'), (152, 163, 'AMEN'), (138, 150, 'AMEN'), (233, 247, 'AMEN'), (240, 256, 'AMEN'), (165, 179, 'AMEN'), (203, 213, 'AMEN')]}
Skipping entity
hotel fine decorated 21 room providing amenity wifi internet 32 tv telephone electronic safe daily newspaper bathroom full modern amenity {'entities': [(99, 107, 'AMEN'), (67, 75, 'AMEN'), (93, 107, 'AMEN')]}
hotel feature water purification system free wi fi internet connection car parking travel desk daily house keeping {'entities': [(45, 49, 'AMEN'), (45, 58, 'AMEN')]}
Skipping entity
facility newspaper wifi internet attach balcony parking travel desk car rental day laundry service separate lounge room {'entities': [(9, 17, 'AMEN'), (68, 77, '

 42%|████████████████                      | 984/2336 [00:00<00:00, 1569.93it/s]

...

 {'entities': [(72, 81, 'AMEN'), (83, 91, 'AMEN'), (93, 103, 'AMEN'), (122, 127, 'AMEN'), (122, 136, 'AMEN')]}
Skipping entity
room provided modern amenity like mini bar tea coffee maker lcd tv electronic safe {'entities': [(39, 41, 'AMEN'), (67, 81, 'AMEN'), (34, 41, 'AMEN'), (43, 58, 'AMEN')]}
room equipped air conditioner cable television intercom facility data port request chargeable snack fruit basket {'entities': [(47, 63, 'AMEN'), (47, 54, 'AMEN')]}
hotel offer facility like business centre banquet internet wi fi parking dry cleaning laundry service {'entities': [(86, 100, 'AMEN'), (73, 100, 'AMEN'), (59, 63, 'AMEN')]}
Skipping entity
lcd tv telephone tea coffee maker mini bar electronic safe wi fi room amenity {'entities': [(7, 15, 'AMEN'), (39, 41, 'AMEN'), (54, 57, 'AMEN'), (43, 57, 'AMEN'), (34, 41, 'AMEN'), (17, 32, 'AMEN'), (59, 63, 'AMEN')]}
Skipping entity
Skipping entity
Skipping entity

 82%|██████████████████████████████▎      | 1912/2336 [00:01<00:00, 1444.90it/s]


host modern amenity offered hotel central air conditioning elevator business center central heating parking facility wi fi internet facility laundry doctor call travel desk taxi service room service {'entities': [(59, 66, 'AMEN'), (186, 197, 'AMEN'), (173, 184, 'AMEN'), (42, 57, 'AMEN'), (149, 159, 'AMEN'), (100, 115, 'AMEN'), (161, 171, 'AMEN'), (123, 139, 'AMEN'), (117, 130, 'AMEN')]}
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
spacious well decorated room hdtv cable connection tea coffee maker hair dryer mini bar ironing board wi fi internet room safe {'entities': [(84, 86, 'AMEN'), (88, 91, 'AMEN'), (102, 115, 'AMEN'), (79, 86, 'AMEN'), (88, 100, 'AMEN'), (51, 66, 'AMEN')]}
room fitted contemporary amenity like satellite lcd tv tea coffee maker electronic safe mini bar wi fi internet connectivity {'entities': [(93, 95, 'AMEN'), (83, 86, 'AMEN'), (72, 86, 'AMEN'), (88, 95, 'AMEN'), (55, 70, 'AMEN')]}
offer range guest facility like room service p

100%|█████████████████████████████████████| 2336/2336 [00:01<00:00, 1558.78it/s]


Skipping entity
Skipping entity
lcd tv telephone tea coffee maker mini bar electronic safe wi fi room amenity {'entities': [(7, 15, 'AMEN'), (39, 41, 'AMEN'), (43, 57, 'AMEN'), (34, 41, 'AMEN'), (17, 32, 'AMEN')]}
hotel provides facility like business centre board room banquet travel assistance wi fi internet access wake call service parking facility {'entities': [(64, 80, 'AMEN'), (45, 54, 'AMEN'), (85, 95, 'AMEN'), (82, 95, 'AMEN')]}
room facility include air conditioner cable television intercom facility mini bar tea coffee maker refrigerator fireplace bathroom amenity {'entities': [(78, 80, 'AMEN'), (73, 80, 'AMEN'), (112, 120, 'AMEN'), (55, 71, 'AMEN'), (82, 97, 'AMEN')]}
modern facility like work desk safe mini bar heat convector dvd player cable tv provided room {'entities': [(41, 43, 'AMEN'), (36, 43, 'AMEN')]}
Skipping entity
resort feature well appointed spacious room fitted satellite television study table telephone tea coffee maker fully stocked minibar {'entities': [(84, 9

#### After creating data in the specific format to train spacy custom ner model , I have followed the following steps in the below documentation to train the model  : https://spacy.io/usage/training , https://medium.com/analytics-vidhya/custom-named-entity-recognition-ner-model-with-spacy-3-in-four-steps-7e903688d51

#### After Training a model folder is created which could be then loaded in to find the custom entities given a description